In [121]:
# Render our plots inline
%matplotlib inline
%pylab inline  
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

# General Plotting Parameters
mpl.rcParams['figure.figsize'] = (8,5)
mpl.rcParams['lines.linewidth'] = 2.5
mpl.rcParams['font.weight'] = 'bold'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['font.size'] = 14.
mpl.rcParams['legend.fontsize'] = 12.
mpl.rcParams['axes.labelsize'] = 12.
mpl.rcParams['xtick.labelsize'] = 10.
mpl.rcParams['ytick.labelsize'] = 10.
mpl.rcParams['xtick.minor.pad'] = 4
mpl.rcParams['xtick.direction'] = 'out'
mpl.rcParams['ytick.direction'] = 'out'

#Git says this is patched, but it doesn't work from Pip --upgrade 26-mar-2015
#mpl.rcParams['xtick.minor.visible'] = True  

# These are the "Tableau 20" colors as RGB.  
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14),
             (255, 187, 120), (44, 160, 44), (152, 223, 138),
              (148, 103, 189),
             (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127),
             (199, 199, 199), (188, 189, 34), (219, 219, 141),
             (23, 190, 207), (158, 218, 229),(214, 39, 40), (255, 152, 150)]  
    
# Scale the RGB values to the [0, 1] range,
# which is the format matplotlib accepts.  
for i in range(len(tableau20)): 
    r, g, b = tableau20[i]  
    tableau20[i] = (r / 255., g / 255., b / 255.)  

Populating the interactive namespace from numpy and matplotlib


In [122]:
import calc_enrich
reload(calc_enrich)
from calc_enrich import calc_del_U
from calc_enrich import stages_per_cascade
from calc_enrich import N_product_by_alpha
from calc_enrich import N_waste_by_alpha
from calc_enrich import machines_per_enr_stage
from calc_enrich import machines_per_strip_stage
from calc_enrich import product_per_enr_stage
from calc_enrich import waste_per_strip_stage
from calc_enrich import Npc_from_Nstages
from calc_enrich import Nwc_from_Nstages
from calc_enrich import machines_per_cascade
from calc_enrich import allowed_feed_per_stage
from calc_enrich import calc_feed_flows

In [123]:
# centrifuge params 
#omega = 64000
#d = 0.1  # m 
#Z = 2.0   # m
#F_m_hrs = 70 # grams/hr  
#T = 320.0   # K
#cut = 0.5
eff = 1.0 # centrifuge efficiency

d = 0.15  # m 
Z = 1.0   # m
F_m_hrs = 15*60*60/(1e3) # grams/hr  
T = 320.0# K
cut = 0.5
omega = 485/(d/2)

# cascade params
Nfc = 0.007
Npc = 0.035
Nwc = 0.003
Fc_month = 739 #kg/month

# The feed enrichment to be used when using a cascade designed for LEU to produce HEU.
Nfc_mod = 0.035
Fc_month_mod = Fc_month #kg/month
#Fc_month_mod = 46 #kg/month

#unit conversions
kg_sec2kg_month = 30.4*24*60*60
v_a = omega * (d/2.0)
F_m = F_m_hrs/(60*60*1000.0)
Fc = Fc_month/kg_sec2kg_month
Fc_mod = Fc_month_mod/kg_sec2kg_month

# Comparison numbers (not needed for calculations)
del_U_th_yr = 1.1 #swu/yr
del_U_obs_yr = 0.71 #Swu/yr
del_U_th = del_U_th_yr/(365.25*24*60*60) #kgSWU/sec
del_U_obs = del_U_obs_yr/(365.25*24*60*60)

Pc_month = 77 #kg/month
Pc = Pc_month/kg_sec2kg_month

qty_len = 6
assay_len = 3

In [124]:
alpha, del_U, del_U_yr = calc_del_U(v_a, Z, d, F_m, T, cut, eff)
ideal_enrich_s, ideal_strip_s= stages_per_cascade(alpha, Nfc, Npc, Nwc)
print "del_U is", del_U, " alpha is ", alpha

feed_flows = calc_steady_flow(int(round(ideal_enrich_s)), int(round(ideal_strip_s)), Fc, cut)
print int(round(ideal_enrich_s)), int(round(ideal_strip_s))
feed_flows


del_U is 3.1939942861e-07  alpha is  1.34782845204
5 2


array([ 0.0003517 ,  0.00070339,  0.00105509,  0.00084407,  0.00063305,
        0.00042203,  0.00021102])

In [136]:

Nfs = Nfc
print "Stage   #Mach   Feed\t    Product\t Waste\t   F_assay \tP_assay W_assay"
all_stages = []
for stage in range(int(round(ideal_enrich_s))):
    curr_stage = stage + int(round(ideal_strip_s))
    Fs = feed_flows[curr_stage]
    n_mach_enr = round(machines_per_enr_stage(alpha, del_U, Fs))
    if (n_mach_enr <= 1):
        int_mach = False
    Nps = N_product_by_alpha(alpha, Nfs)
    Ps = product_per_enr_stage(alpha, Nfs, Nps, Fs)
    Ws = Fs - Ps
    Nws = N_waste_by_alpha(alpha, Nfs)
    enrich_stages.append([stage, n_mach_enr, Fs, Ps, Ws])
    if (stage == 0):
        W_enr1 = Ws
        Nw_1 = Nws
    print stage, "\t", n_mach_enr,"\t", round(Fs,qty_len), "  ",round(Ps, qty_len), "  ",round(Ws, qty_len),"  ", round(Nfs, assay_len), "\t",round(Nps, assay_len),"\t", round(Nws, assay_len)
    Nfs = Nps



Stage   #Mach   Feed	    Product	 Waste	   F_assay 	P_assay W_assay
0 	200.0 	0.001055    0.000529    0.000526    0.007 	0.009 	0.005
1 	160.0 	0.000844    0.000423    0.000421    0.009 	0.013 	0.007
2 	120.0 	0.000633    0.000318    0.000315    0.013 	0.017 	0.009
3 	80.0 	0.000422    0.000212    0.00021    0.017 	0.023 	0.013
4 	40.0 	0.000211    0.000106    0.000105    0.023 	0.03 	0.017


In [141]:

Nfs = Nw_1
print "Stage   #Mach   Feed\t    Product\t Waste\t   F_assay \tP_assay W_assay"

for stage in range(int(round(ideal_strip_s))):
    curr_stage = stage
    Fs = feed_flows[curr_stage]
    n_mach_strip = round(machines_per_strip_stage(alpha, del_U, Fs))
    if (n_mach_enr <= 1):
        int_mach = False
    Nps = N_product_by_alpha(alpha, Nfs)
    Ws = waste_per_strip_stage(alpha, Nfs, Nws, Fs)
    Ps = Fs - Ws
    Nws = N_waste_by_alpha(alpha, Nfs)
    strip_stages.append([-1*(stage+1), n_mach_strip, Fs, Ps, Ws])
    print -1*(stage+1), "\t",n_mach_strip ,"\t", round(Fs,qty_len), "  ",round(Ps, qty_len), "  ",round(Ws, qty_len),"  ", round(Nfs, assay_len), "\t",round(Nps, assay_len),"\t", round(Nws, assay_len)
    Nfs = Nws


## NOT CORRECT ##

Stage   #Mach   Feed	    Product	 Waste	   F_assay 	P_assay W_assay
-1 	67.0 	0.000352    0.000216    0.000136    0.005 	0.007 	0.004
-2 	133.0 	0.000703    -inf    inf    0.004 	0.005 	0.003


In [127]:
   
    
epsilon = alpha - 1.0
F_strip = W_enr1
print "Feed for strip (kg/mon)", F_strip*kg_sec2kg_month
# Consider the zeroth stage of cascade to be on enrich side (iterate from 0), 
# then iterate from 1 for strip side
print "stage, #mach,       Feed,    W_strip,   Nf_strip,   Nw_strip"
int_stages = True
while (int_stages == True):
    n_mach_s= round(machines_per_strip_stage(alpha, del_U, F_strip))
    if (n_mach_s <= 1):
        int_stages = False
    Nw_strip = N_waste_by_alpha(alpha, Nf_strip)
    W_strip = waste_per_strip_stage(alpha, Nf_strip, Nw_strip, F_strip)
    strip_stages.append([-1*(n_stage_str+1),n_mach_s])
    print n_stage_str, n_mach_s, F_strip, W_strip, Nf_strip, Nw_strip
    F_strip = W_strip
    Nf_strip = Nw_strip
    n_stage_str+=1

print "waste of cascade (kg/mon)" , W_strip*kg_sec2kg_month


Feed for strip (kg/mon) 184.593536625
stage, #mach,       Feed,    W_strip,   Nf_strip,   Nw_strip


NameError: name 'Nf_strip' is not defined